# Silver Dim_customer

**Tipo de tabla:** Maestro de clientes

**Origen:** `customer`

**Destino:** `dim_customer`

## Lectura de datos parquet de Bronze

In [1]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

load_dotenv("/home/jovyan/work/.env")
spark = SparkSession.builder.appName("silver_customer").getOrCreate()

Se genera nuevo Dataframe de spark desde tablas bronze.

In [2]:
bronze_path = os.getenv("BRONZE_PATH")
silver_path = os.getenv("SILVER_PATH")

directory_path = os.path.join(bronze_path, "customer")
bronze_df = spark.read.parquet(directory_path)

# Display esquema y filas
bronze_df.printSchema()
bronze_df.show(5)


root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- canal: string (nullable = true)
 |-- segmento_actual: string (nullable = true)
 |-- region: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- fecha_alta: timestamp (nullable = true)
 |-- estado: string (nullable = true)
 |-- mix_credito_vs_contado_3m: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_file: string (nullable = true)

+-----------+-------------+-----------+---------------+--------------------+-----------+--------------------+--------+-------------------------+--------------------+--------------------+
|customer_id|customer_name|      canal|segmento_actual|              region|     comuna|          fecha_alta|  estado|mix_credito_vs_contado_3m| ingestion_timestamp|         source_file|
+-----------+-------------+-----------+---------------+--------------------+-----------+--------------------+--------+----------------

In [3]:
#Creacion de tabla de paso para limpieza
# esto para utilziar SQL en los procesos de estandarización
bronze_df.createOrReplaceTempView("temp_dim_customer")

## Limpieza de Silver
### Reglas de Limpieza aplicada:
* Tipos Correctos
* Eliminacion de Duplicados por customer_id
* Evitar * en selects

### Reglas de Transformacion:
* Se agrega dato dummy para consistencia en productos no existentes
* Estandarizacion en lenguaje de nombres de columnas

In [4]:
# query de limpieza de datos para silver
query = """
SELECT  COALESCE(customer_id, 0) AS customer_id
        , customer_name
        , channel
        , actual_segment
        , region
        , comuna
        , entry_date
        , status
        , mix_credito_vs_contado_3m
FROM (
    SELECT CAST(customer_id AS INT) AS  customer_id
        , TRIM(CAST(customer_name AS VARCHAR(50))) AS customer_name
        , TRIM(CAST(canal AS VARCHAR(50))) AS channel
        , TRIM(CAST(segmento_actual AS VARCHAR(5))) AS actual_segment
        , TRIM(CAST(region AS VARCHAR(100))) AS region
        , TRIM(CAST(comuna AS VARCHAR(100))) AS comuna
        , CAST(fecha_alta AS TIMESTAMP) AS entry_date
        , TRIM(CAST(estado AS VARCHAR(30))) AS status
        , CAST(mix_credito_vs_contado_3m AS DOUBLE) AS mix_credito_vs_contado_3m
        , ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY fecha_alta DESC) as RW
    FROM temp_dim_customer
)
WHERE RW = 1

UNION ALL

SELECT 0 AS customer_id
    , 'Sin Cliente' AS customer_name
    , 'Sin Canal' AS channel
    , 'N/A'  AS actual_segment
    , 'N/A' AS region
    , 'N/A' AS comuna
    , NULL AS entry_date
    , NULL AS state
    , NULL AS mix_credito_vs_contado_3m
ORDER BY customer_id
"""

# Execute the SQL query and get the result as a new DataFrame
sql_result_df = spark.sql(query)

# Display the results
sql_result_df.printSchema()
sql_result_df.show()

root
 |-- customer_id: integer (nullable = false)
 |-- customer_name: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- actual_segment: string (nullable = true)
 |-- region: string (nullable = true)
 |-- comuna: string (nullable = true)
 |-- entry_date: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- mix_credito_vs_contado_3m: double (nullable = true)

+-----------+-------------+-----------+--------------+--------------------+------------+--------------------+--------+-------------------------+
|customer_id|customer_name|    channel|actual_segment|              region|      comuna|          entry_date|  status|mix_credito_vs_contado_3m|
+-----------+-------------+-----------+--------------+--------------------+------------+--------------------+--------+-------------------------+
|          0|  Sin Cliente|  Sin Canal|           N/A|                 N/A|         N/A|                NULL|    NULL|                     NULL|
|          1|Cliente_00

## Escritura de datos
* Todo se escribe en parquet, en carpetas de silver.
* Se escribe con metodo upsert para posteriores ingestas masivas de datos
* Posibilidad de realizar SCD para preservar cambios historicos en dimensiones

**Mejora** utilziar deltas tables para control de merges y log (ACID)

In [5]:
# Escritura de dim_product en silver como parquet
output_path =  os.path.join(silver_path, "dim_customer.parquet")

# upsert_parquet(
#     spark
#     , sql_result_df       # datos nuevos / actualizados
#     , output_path        # ruta donde están los parquet "históricos"
#     , ['customer_id']     # claves de negocio para el upsert
# )

sql_result_df.write.mode("overwrite").parquet(output_path)

#en caso de necesitar, se puede cambiar logica a SCD para preservar historia

## Validaciones.

In [6]:
# 1. filas origen = filas destino
n_rows_silver= sql_result_df.count() # se le resta el dato dummy
n_rows_bronze= bronze_df.count()
n_rows_silver_parquet = spark.read.parquet(output_path).count()

print(f'Cantidad Filas Silver: {n_rows_silver}')
print(f'Cantidad Filas Bronze: {n_rows_bronze}')
print(f'Cantidad Filas Silver Parquet: {n_rows_silver_parquet}')

if n_rows_silver  - 1 != n_rows_bronze:
    raise Exception("Error Validacion, filas cargadas no son iguales")
elif n_rows_silver != n_rows_silver_parquet:
    raise Exception("Error Validacion, filas de dim no son iguales al origen")
else:
    print(f'Tabla con datos validados.')


Cantidad Filas Silver: 10927
Cantidad Filas Bronze: 10926
Cantidad Filas Silver Parquet: 10927
Tabla con datos validados.


In [7]:
summary_stats = spark.read.parquet(output_path).describe()
summary_stats.show()

+-------+-----------------+-------------+-----------+--------------+-----------+------------+--------+-------------------------+
|summary|      customer_id|customer_name|    channel|actual_segment|     region|      comuna|  status|mix_credito_vs_contado_3m|
+-------+-----------------+-------------+-----------+--------------+-----------+------------+--------+-------------------------+
|  count|            10927|        10927|      10927|         10927|      10927|       10927|   10926|                    10926|
|   mean|           5463.0|         NULL|       NULL|          NULL|       NULL|        NULL|    NULL|      0.34664309766015383|
| stddev|3154.497529982654|         NULL|       NULL|          NULL|       NULL|        NULL|    NULL|       0.2347852027502133|
|    min|                0|Cliente_00001|  Sin Canal|             A|Antofagasta| Antofagasta|  activo|                      0.0|
|    max|            10926|  Sin Cliente|tradicional|           N/A| Valparaíso|Viña del Mar|inac